In [ ]:
!pip3 install pandas
!pip3 install folium
!pip3 install dask
!pip3 install "dask[distributed]" --upgrade
!pip3 install aiohttp

In [ ]:
import pandas as pd
import folium
import requests
import os
import dask
from time import sleep
from dask import delayed
from dask.distributed import Client
import dask.dataframe as dd

In [ ]:
df_station = pd.read_csv("./isd-history.csv")
df_station
open_station = df_station[df_station["END"] == 20220502]

In [ ]:
#df = pd.read_csv("data/2022/72011354829.csv")
#df.dtypes

In [ ]:
dtypes = {
'STATION':         int,
'DATE':           object,
'SOURCE':         object,
'LATITUDE':      float,
'LONGITUDE':     float,
'ELEVATION':     float,
'NAME':           object,
'REPORT_TYPE':    object,
'CALL_SIGN':      object,
'QUALITY_CONTROL':object,
'WND':            object,
'CIG':            object,
'VIS':            object,
'TMP':            object,
'DEW':            object,
'SLP':            object,
'AA1':            object,
'AT1':            object,
'AT2':            object,
'AT3':            object,
'AT4':            object,
'AT5':            object,
'AU1':            object,
'AW1':            object,
'OC1':            object,
'REM':            object,
'EQD':            object
}

In [ ]:
client = Client(n_workers=4)

In [ ]:
if not os.path.exists("./data"):
    os.mkdir("./data")

for year in range(2022, 2023):
    year = str(year)
    path = "./data/" + year

    if not os.path.exists(path):
        os.mkdir(path)

    for key, station_row in open_station.iterrows():
        wban = str(station_row.WBAN);

        while (len(wban) < 5):
            wban = '0' + wban

        station  = str(station_row.USAF) + wban

        if not os.path.exists(path + "/" + station + ".csv"):
            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

            if (requests.get(url).status_code == 200):
                #df = pd.read_csv(url)
                ddf = dd.read_csv(url, dtype='object')
                ddf.to_csv(path + "/" + station + ".csv", index=True, single_file = True)
                #df = pd.read_csv(url).to_csv(path + "/" + station + ".csv", index=True)
                #sleep(1)

In [ ]:


for year in range(2022, 2023):
    year = str(year)
    path = "./data/" + year

    for key, station_row in open_station.iterrows():
        wban = str(station_row.WBAN);

        while (len(wban) < 5):
            wban = '0' + wban

        station  = str(station_row.USAF) + wban

        if os.path.exists(path + "/" + station + ".csv"):
            #df = pd.read_csv(path + "/" + station + ".csv")
            #print(df.dtypes)
            df = dd.read_csv(path + "/" + station + ".csv", dtype=dtypes)
            
            
            #df = dd.read_csv(path + "/" + station + ".csv")
            #df.head()


# Clean dataframe

In [ ]:
big_frame['TMP_control'] = big_frame['TMP'].str.split(pat=',', expand=True)[1]
big_frame['TMP_test'] = big_frame['TMP'].str.split(pat=',', expand=True)[0]
big_frame['TMP_sign'] = big_frame['TMP'].astype(str).str[0]
big_frame['TMP_final'] = (big_frame['TMP_test'].astype(str).str[1:4] + "." + big_frame['TMP_test'].astype(str).str[4:]).astype(float)
big_frame['DATE'] = pd.to_datetime(big_frame['DATE'], format="%Y-%m-%dT%H:%M:%S")
big_frame = big_frame[big_frame['TMP_control'] == "1"]

In [ ]:
big_frame['TMP_sign'].replace({"+": 1, "-": -1}, inplace=True)
big_frame['TMP_final'] = big_frame['TMP_sign'] * big_frame['TMP_final']
big_frame

In [ ]:
cleaned_frame = big_frame[["NAME", "TMP_final", "LATITUDE", "LONGITUDE", "DATE"]]
cleaned_frame

# Statistiques globales de température (moy, min, max)

In [ ]:
temp = cleaned_frame.groupby(by="NAME").agg({'TMP_final': ['mean', 'min', 'max'], "LATITUDE": "first", "LONGITUDE": "first"})
temp

In [ ]:
#for year in range(2022, 2023):
#    year = str(year)
#    path = "./data/" + year

#    if not os.path.exists(path):
#        os.mkdir(path)

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban

#        if not os.path.exists(path + "/" + station + ".csv"):
#            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

#            if (requests.get(url).status_code == 200):
#                pd.read_csv(url).to_csv(path + "/" + station + ".csv")

#            spark.sparkContext.addFile('https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv')

#            if (SparkFiles.get(station + ".csv")):
#                df = spark.read.csv("file://"+SparkFiles.get(station + ".csv"), header=True)

#                print("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")
#                df.write.format("csv").save("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")

#for year in range(2022, 2023):

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban
        
#        df = spark.read.csv("hdfs://namenode:9000/noaa/" + str(year) + "/" + station + ".csv", header=True)
#df = spark.read.csv("hdfs://namenode:9000/noaa/2022/72026853882.csv", header=True)
#df.take(5)